In [70]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np


# Enable eager execution (required for .numpy() calls)
tf.config.run_functions_eagerly(True)


In [71]:

# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "C:/College/3rd-Year/codes/Datasets-ANN/Cat-Dog/train"  # e.g. '
test_dir = "C:/College/3rd-Year/codes/Datasets-ANN/Cat-Dog/test"    # e.g. '

In [72]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [73]:
import os
os.path.exists(test_dir)

True

In [74]:
import os

folder = "C:/College/3rd-Year/codes/Datasets-ANN/Cat-Dog/train"
print(os.listdir(folder))

['Cat', 'Dog']


In [75]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 20000 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.
Found 24998 images belonging to 2 classes.


In [76]:
import os

root = train_dir  # or r"C:\path\to\train_1"
bad_files = []

valid_ext = (".jpg", ".jpeg", ".png", ".bmp", ".gif")

for r, d, f_list in os.walk(root):
    for f in f_list:
        if not f.lower().endswith(valid_ext):
            bad_files.append(os.path.join(r, f))

bad_files

[]

In [77]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    # layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    # layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    # layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [78]:

# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [79]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=val_generator
)

625/625 ━━━━━━━━━━━━━━━━━━━━ 300s 480ms/step - accuracy: 0.6445 - loss: 0.6234 - val_accuracy: 0.6615 - val_loss: 0.6069


In [80]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 111s 142ms/step - accuracy: 0.6689 - loss: 0.5975
Test Accuracy: 0.6688535213470459


In [81]:

# SAVE THE MODEL
model.save('exercise_6_custom_Amparo_Solito.h5')

In [82]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_custom_Amparo_Solito.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Cat" if pred >= 0.5 else "Dog"
    print(f"Prediction: {label} (confidence: {pred:.2f})")

In [83]:
# Example usage:
predict_image("C:/College/3rd-Year/codes/Datasets-ANN/Cat-Dog/test/Dog/0.jpg")
predict_image("C:/College/3rd-Year/codes/Datasets-ANN/Cat-Dog/test/Cat/0.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step


Prediction: Cat (confidence: 0.73)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
Prediction: Cat (confidence: 0.95)


In [84]:
from tensorflow.keras import regularizers

# L2 regularization factor
l2_factor = 0.001

model_improved = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l2_factor),
                  input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l2_factor)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l2_factor)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2_factor)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile with the same optimizer and learning rate schedule
model_improved.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [85]:
# Create a new optimizer instance with the same learning rate schedule
optimizer_improved = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Compile the improved model with the new optimizer
model_improved.compile(optimizer=optimizer_improved, 
                       loss='binary_crossentropy', 
                       metrics=['accuracy'])

In [ ]:
history_improved = model_improved.fit(
    train_generator,
    epochs=1,  # you can increase if needed
    validation_data=val_generator
)


Epoch 1/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 343s 549ms/step - accuracy: 0.5300 - loss: 0.7474 - val_accuracy: 0.5784 - val_loss: 0.6919
Epoch 2/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 489s 784ms/step - accuracy: 0.5700 - loss: 0.6914 - val_accuracy: 0.5728 - val_loss: 0.6861


In [90]:
test_loss_improved, test_acc_improved = model_improved.evaluate(test_generator)
print(f"Improved CNN Test Accuracy: {test_acc_improved:.4f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 51s 65ms/step - accuracy: 0.5709 - loss: 0.6865
Improved CNN Test Accuracy: 0.5709


In [92]:
def predict_image_improved(img_path, model=model_improved):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")

# Run predictions

predict_image("C:/College/3rd-Year/codes/Datasets-ANN/Cat-Dog/test/Dog/0.jpg")
predict_image("C:/College/3rd-Year/codes/Datasets-ANN/Cat-Dog/test/Cat/0.jpg")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Prediction: Cat (confidence: 0.73)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Prediction: Cat (confidence: 0.95)


In [ ]:
model_improved.save("exercise_6_custom_enhanced.h5")
